In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
meta_df = pd.read_csv("ver_laps_silverstone.csv")

In [11]:
meta_df['LapTime'] = pd.to_timedelta(meta_df['LapTime'])

In [33]:
compound_ohe = pd.get_dummies(meta_df['Compound'].astype(int), prefix='compound')
for i in range(5):
    col = f'compound_{i}'
    if col not in compound_ohe.columns:
        compound_ohe[col] = 0
compound_ohe = compound_ohe[[f'compound_{i}' for i in range(5)]]

meta_df = pd.concat([meta_df, compound_ohe], axis=1)


In [35]:
def normalize(col):
    return (col - col.min()) / (col.max() - col.min())

In [38]:
meta_df['TyreLife_norm'] = normalize(meta_df['TyreLife'])
meta_df['AirTemp_norm'] = normalize(meta_df['AirTemp'])
meta_df['TrackTemp_norm'] = normalize(meta_df['TrackTemp'])

In [39]:
meta_df.head()

,LapTime,LapNumber,Compound,TyreLife,TrackStatus,AirTemp,TrackTemp,Session,Year,track_stat_1,track_stat_12,track_stat_21,compound_0,compound_1,compound_2,compound_3,compound_4,TyreLife_norm,AirTemp_norm,TrackTemp_norm
0,0 days 00:01:30.274000,2.0,2,2.0,1,24.6,43.2,2021 Season Round 10: British Grand Prix - Pra...,2021,True,False,False,False,False,True,False,0,0.000000,0.84375,0.824773
1,0 days 00:01:30.459000,4.0,2,4.0,1,24.6,46.1,2021 Season Round 10: British Grand Prix - Pra...,2021,True,False,False,False,False,True,False,0,0.066667,0.84375,0.912387
2,0 days 00:01:28.179000,7.0,1,2.0,1,25.1,43.7,2021 Season Round 10: British Grand Prix - Pra...,2021,True,False,False,False,True,False,False,0,0.000000,0.87500,0.839879
3,0 days 00:01:27.745000,9.0,1,4.0,1,25.3,45.5,2021 Season Round 10: British Grand Prix - Pra...,2021,True,False,False,False,True,False,False,0,0.066667,0.88750,0.894260
4,0 days 00:01:27.035000,12.0,0,2.0,1,25.5,47.2,2021 Season Round 10: British Grand Prix - Pra...,2021,True,False,False,True,False,False,False,0,0.000000,0.90000,0.945619


In [27]:
meta_df = meta_df.drop(columns=['compound_0','compound_1','compound_2', 'compound_3', 'compound_4'])

In [8]:
meta_arr = meta_df.to_numpy()

In [9]:
meta_arr.shape

(289, 9)

In [52]:
x = meta_df.drop(columns=[
    'LapTime',
    'LapNumber',
    'Compound',
    'TyreLife',
    'TrackStatus',
    'AirTemp',
    'TrackTemp',
    'Session',
    'Year']).to_numpy()

In [54]:
x

array([[True, False, False, ..., 0.0, 0.8437500000000001,
        0.8247734138972811],
       [True, False, False, ..., 0.06666666666666667, 0.8437500000000001,
        0.9123867069486405],
       [True, False, False, ..., 0.0, 0.8750000000000001,
        0.83987915407855],
       ...,
       [True, False, False, ..., 0.23333333333333334, 0.50625,
        0.2567975830815709],
       [True, False, False, ..., 0.26666666666666666, 0.5187499999999999,
        0.2567975830815709],
       [True, False, False, ..., 0.3, 0.525, 0.2628398791540786]],
      shape=(289, 11), dtype=object)

In [42]:
y = meta_df['LapTime'].dt.total_seconds().to_numpy()

In [44]:
y

array([ 90.274,  90.459,  88.179,  87.745,  87.035,  92.76 ,  92.414,
        92.641,  92.793,  92.405,  92.321,  90.428,  91.642,  91.351,
        91.3  ,  91.236,  91.068,  91.178,  93.699,  90.822,  90.404,
        89.902,  91.685,  92.445,  92.088,  91.787,  91.707,  91.624,
        86.751,  86.504,  86.315,  86.306,  86.209,  91.438,  90.488,
        89.885,  89.149,  94.789,  93.226,  93.611,  93.592,  88.541,
        88.498,  88.386,  87.901,  87.984, 105.743, 102.814, 100.452,
        99.975,  99.129, 101.486, 100.655, 104.887, 102.996, 101.055,
       101.145,  93.807,  93.489,  93.539,  93.762,  93.451,  93.606,
        93.486,  93.632,  94.388,  94.806,  95.015,  94.994,  94.973,
        94.996,  94.87 ,  94.963,  94.826,  95.405,  94.441,  94.67 ,
        94.482,  94.404,  94.488,  94.171,  94.415,  94.503,  94.21 ,
        94.068,  94.924,  94.023,  93.322,  92.354,  92.135,  92.516,
        92.584,  92.566,  92.964,  93.051,  92.858,  93.581,  90.443,
        89.863,  93.

x_telemetry = np.load('ver_silverstone_telemetry_data.npy', allow_pickle = True)

In [57]:
x_telemetry.shape

(289, 150, 3)

In [58]:
from tqdm import tqdm
num_laps, time_steps, telemetry_features = x_telemetry.shape
meta_features = x.shape[1]

x_combined = np.zeros((num_laps, time_steps, telemetry_features + meta_features))

for i in tqdm(range(num_laps), desc="Appending metadata"):
    telemetry = x_telemetry[i]
    meta = x[i]

    meta_repeated = np.tile(meta, (time_steps, 1))

    x_combined[i] = np.concatenate([telemetry, meta_repeated], axis=1)

Appending metadata: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 289/289 [00:00<00:00, 12283.06it/s]


In [59]:
x_combined.shape

(289, 150, 14)

In [60]:
np.save('x_combined.npy', x_combined)
np.save('y.npy', y)